In [1]:
% matplotlib inline
from scipy.stats import invgamma
from scipy.stats import norm
from multiprocessing import Pool
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
def integral(estimate, ts):
    elements = np.ones(len(ts) - 1)
    for i in range(len(ts) - 1):
        elements[i] = (ts[i+1] - ts[i])*(estimate[i+1] + estimate[i])/2
    return sum(elements)

def sum1(beta):
    return np.sum(p[:, 0] - beta * p[:, 3])

def sum2(alpha):
    return np.sum(p[:, 3] * (p[:, 0] - alpha))

def sum3(alpha, beta):
    l = (p[:, 0] - alpha - beta*p[:, 3])
    return np.sum(l * l )

def loglike(alpha, beta, sigma):
    return N*np.log(1/(np.sqrt(2*np.pi*sigma))) - sum3(alpha, beta) / (2*sigma)

def function1(u):
    alphas = np.ones(sample_iter)
    betas = np.ones(sample_iter)
    sigmas = np.ones(sample_iter)
    estimates = np.ones(n+1)
    
    if u < m:
        r = np.random.RandomState(u)

        for i in range(n+1):
            t = ts[i]
            
            if i == 0:
                alphas[0] = 3000
                betas[0] = 185
                sigmas[0] = 90000

            else:
                alphas[0] = np.mean(alpha_sample)
                betas[0] = np.mean(beta_sample)
                sigmas[0] = np.mean(sigma_sample)

            for j in range(sample_iter -1):

                location_alpha = (sigma_alpha*t*sum1(betas[j]) + sigmas[j]*mu_alpha) / (sigma_alpha * N*t + sigmas[j])
                scale_alpha = np.sqrt((sigma_alpha * sigmas[j]) / (sigma_alpha * N*t + sigmas[j]))
                alphas[j+1] = r.normal(loc = location_alpha, scale = scale_alpha)

                location_beta = (sigma_beta * t * sum2(alphas[j+1]) + sigmas[j] * mu_beta) / (sigma_beta *t* ssx + sigmas[j])
                scale_beta = np.sqrt((sigmas[j] * sigma_beta) / (sigma_beta *t* ssx + sigmas[j]))
                betas[j+1] = r.normal(loc = location_beta, scale = scale_beta)

                shape = N*t/2 + a
                invrate = 2*b / (b*t*sum3(alphas[j+1], betas[j+1]) + 2)
                sigmas[j+1] = invgamma.rvs(a = shape, scale = 1/ invrate, random_state = r)

            alpha_sample = alphas[burn_in:]
            beta_sample = betas[burn_in:len(betas)]
            sigma_sample = sigmas[burn_in:len(sigmas)]

            box = np.ones(len(alpha_sample))
            for k, l in enumerate(alpha_sample):
                box[k] = loglike(l, beta_sample[k], sigma_sample[k])
                
            estimates[i] = np.average(box)
    
    #print(estimates)
    return estimates
    
def sum4(beta):
    return np.sum(p[:, 0] - beta * p[:, 4])

def sum5(alpha):
    return np.sum(p[:, 4] * (p[:, 0] - alpha))

def sum6(alpha, beta):
    l = (p[:, 0] - alpha - beta*p[:, 4])
    return np.sum(l * l )  

def loglike2(alpha, beta, sigma):
    return N*np.log(1/(np.sqrt(2*np.pi*sigma))) - sum6(alpha, beta) / (2*sigma)
    
def function2(w):
    gammas = np.ones(sample_iter)
    deltas = np.ones(sample_iter)
    taus = np.ones(sample_iter)
    estimates = np.ones(n+1)
    
    if w < m:
        r = np.random.RandomState(w)

        for i in range(n+1):
            t = ts[i]
            
            if i == 0:
                gammas[0] = 3000
                deltas[0] = 185
                taus[0] = 90000

            else:
                gammas[0] = np.mean(gamma_sample)
                deltas[0] = np.mean(delta_sample)
                taus[0] = np.mean(tau_sample)

            for j in range(sample_iter - 1):

                location_alpha = (sigma_alpha*t*sum4(deltas[j]) + taus[j]*mu_alpha) / (sigma_alpha * N*t + taus[j])
                scale_alpha = np.sqrt((sigma_alpha * taus[j]) / (sigma_alpha * N*t + taus[j]))
                gammas[j+1] = r.normal(loc = location_alpha, scale = scale_alpha)

                location_beta = (sigma_beta * t * sum5(gammas[j+1]) + taus[j] * mu_beta) / (sigma_beta *t* ssz + taus[j])
                scale_beta = np.sqrt((taus[j] * sigma_beta) / (sigma_beta *t* ssz + taus[j]))
                deltas[j+1] = r.normal(loc = location_beta, scale = scale_beta)

                shape = N*t/2 + a
                invrate = 2*b / (b*t*sum6(gammas[j+1], deltas[j+1]) + 2)
                taus[j+1] = invgamma.rvs(a = shape, scale = 1/ invrate, random_state = r)

            gamma_sample = gammas[burn_in:]
            delta_sample = deltas[burn_in:len(deltas)]
            tau_sample = taus[burn_in:len(taus)]

            box2 = np.ones(len(gamma_sample))
            for k, l in enumerate(gamma_sample):
                box2[k] = loglike2(l, delta_sample[k], tau_sample[k])

            estimates[i] = np.average(box2)
        
    return estimates

In [25]:
pine = pd.read_table("pine.txt", delim_whitespace = True)
p = pine.values
pine['ave_x'] = pine['x'] - np.average(p[:, 1])
pine['ave_z'] = pine['z'] - np.average(p[:, 2])
p = pine.values

mu_alpha = 3000
sigma_alpha = 10**6
mu_beta = 185
sigma_beta = 10**4
a = 3
b = 1/(2*300**2)

N = np.shape(p)[0]
ssx = np.sum(p[:, 3] * p[:, 3])
ssz = np.sum(p[:, 4] * p[:, 4])

n = 10
c = 2
ts = [(i/n)**c for i in range(n+1)]

####ここをいじってね。
sample_iter = 10000
burn_in = 3000
m = 100
####

# computing BF


model1 marginal likelihood by power posterior

In [26]:
result1 = Pool().map(function1, range(m))

In [27]:
result1

[array([-712.90867573, -372.61009972, -328.18147337, -316.25172717,
        -311.28192504, -308.59079483, -307.14042616, -306.14401568,
        -305.53065908, -305.06570075, -304.77933389]),
 array([-731.57933428, -372.64323895, -327.89788689, -316.30742987,
        -311.1736021 , -308.56156683, -307.10796589, -306.18928041,
        -305.5628934 , -305.06322227, -304.74301861]),
 array([-739.05710425, -372.79006872, -328.24501091, -316.36428212,
        -311.17930349, -308.63922202, -307.13520879, -306.16589182,
        -305.54236442, -305.10682422, -304.74921227]),
 array([-717.07005732, -371.74760255, -327.59756307, -316.32364024,
        -311.30811801, -308.53843636, -307.14473181, -306.19427728,
        -305.48172093, -305.08700476, -304.73737777]),
 array([-723.31573513, -371.98300228, -328.16254796, -316.40290505,
        -311.16635796, -308.74129163, -307.16256751, -306.16725593,
        -305.50405166, -305.0618352 , -304.72004214]),
 array([-724.55995481, -371.19487477, -328.01

In [11]:
expect1 = np.ones(100)
for i in range(100):
    expect1[i] = integral(result1[i], ts)

In [12]:
expect1

array([-311.12526488, -311.263866  , -311.50583205, -311.30598936,
       -311.12828261, -311.52443833, -311.33110477, -311.48110953,
       -311.52057671, -311.36185032, -311.5852267 , -311.30480766,
       -311.43216299, -311.49289725, -311.42179548, -311.30554117,
       -311.69247305, -311.45468347, -311.77385862, -311.65929186,
       -311.30080835, -311.45039732, -311.33606779, -311.65959431,
       -311.49742632, -311.46307493, -311.22717102, -311.36711009,
       -311.43859465, -311.67586959, -311.23310943, -311.49909681,
       -311.39446301, -311.32667628, -311.33235538, -311.61180505,
       -311.28548388, -311.55613145, -311.32447681, -311.4891895 ,
       -311.41708389, -311.39832756, -311.37934103, -311.53184868,
       -311.45894056, -311.36534502, -311.44023541, -311.47778371,
       -311.28542542, -311.52274431, -311.43738232, -311.49403111,
       -311.38858093, -311.38803388, -311.45967723, -311.47427651,
       -311.41755364, -311.37510475, -311.25778161, -311.41760

model2 marginal likelihood by power posterior

In [13]:
result2 = Pool().map(function2, range(m))

In [14]:
result2

[array([-674.69827923, -363.58731507, -319.23566903, -307.38039769,
        -302.56250226, -300.08441022, -298.72689154, -297.57703059,
        -297.06319305, -296.51749186, -296.17703802]),
 array([-687.16164872, -361.80931395, -320.02927451, -308.19658969,
        -302.96242482, -300.26203278, -298.67188541, -297.78462988,
        -296.97936519, -296.49591046, -296.22713534]),
 array([-733.29887937, -361.95142827, -319.17635897, -308.04096642,
        -302.90473819, -300.40217785, -298.74018469, -297.71932227,
        -297.04302104, -296.5357745 , -296.24692049]),
 array([-694.08435972, -363.57697916, -320.1518423 , -307.80486715,
        -302.77646885, -300.3856335 , -298.51659472, -297.62170028,
        -296.97860027, -296.59612894, -296.22967932]),
 array([-653.38290857, -364.98184872, -320.596728  , -307.47506493,
        -302.92916499, -300.01489302, -298.69346617, -297.52163981,
        -297.03849854, -296.58263717, -296.27306047]),
 array([-742.52101025, -365.32447072, -319.56

In [28]:
expect2 = np.ones(100)
for i in range(100):
    expect2[i] = integral(result2[i], ts)

In [29]:
expect2

array([-302.61901882, -302.78617275, -303.00393965, -302.81173615,
       -302.62796775, -303.06430497, -302.82589461, -302.98016589,
       -303.02462573, -302.87313549, -303.09433272, -302.81137049,
       -302.94655415, -303.00207119, -302.92747317, -302.82739246,
       -303.21086051, -302.96558214, -303.28469465, -303.17755459,
       -302.81751018, -302.95470062, -302.8498879 , -303.17687279,
       -302.99067597, -302.98411892, -302.73110076, -302.8657179 ,
       -302.96547942, -303.19439218, -302.74751475, -303.02951051,
       -302.92182291, -302.85069949, -302.83966977, -303.11982488,
       -302.81186365, -303.04852189, -302.84223896, -303.02626674,
       -302.93643945, -302.92746138, -302.90761717, -303.05046998,
       -302.95039996, -302.87202186, -302.9526318 , -302.97864131,
       -302.81369641, -303.03860245, -302.94580497, -303.00380235,
       -302.88858344, -302.90508011, -302.97081271, -302.9891426 ,
       -302.92717923, -302.90668096, -302.76362894, -302.94041

BF21

In [19]:
bf_21 = []
for a,b in zip(expect1, expect2):
    bf_21.append(np.exp(b-a))

In [20]:
bf_21

[4945.5628128686149,
 4806.350060063829,
 4924.0783315006802,
 4886.6056925781786,
 4916.3165350658282,
 4722.6877612537937,
 4940.4424232527035,
 4919.4088039358894,
 4894.9090686738127,
 4859.6166477149563,
 4870.2180029562351,
 4882.6196792602814,
 4844.5461101819837,
 4869.8872579462322,
 4886.9434165609628,
 4808.5396640072349,
 4825.2244568659216,
 4861.4952340506525,
 4861.7997753240561,
 4825.8263660592393,
 4833.3649260443881,
 4893.664572563096,
 4847.3133926665405,
 4830.5785302203376,
 4948.0574685109086,
 4812.4231565900191,
 4895.492960959652,
 4921.6158922068344,
 4784.3967762347856,
 4824.5724968356271,
 4844.4775094029646,
 4767.5427139823714,
 4782.1240958139842,
 4798.107244163738,
 4878.9514517610078,
 4875.5108812648341,
 4786.8135097083687,
 4952.3107421687337,
 4828.2426745716784,
 4735.8796437308938,
 4820.5554579099162,
 4773.6485487258187,
 4777.7445053481742,
 4824.0963021464549,
 4956.9236617470842,
 4882.0630006345355,
 4854.2195496829127,
 4910.55574342296

In [21]:
bias = np.average(bf_21) - 4862
std = np.sqrt(np.var(bf_21))

In [22]:
bias

-6.6961979172310748

In [23]:
std

54.929592568557609

In [24]:
np.sqrt(np.sum([(i - 4862)**2 for i in bf_21])/100)

55.336237731657214